# Assignment 2


You must submit your notebook by running `python3 -m autograder.run.submit Assignment2.ipynb` from your local repository.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).

#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWERS HERE", as well as your name below:

## Some Helper Functions

Implement an evaluation function that takes in some board position and player colour and returns a score. 

In [2]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

## Q1

Implement the minimax algorithm that chooses the best move for a given board position, arbitrary evaluation function, player colour, and depth.

Make sure to write your implementation to be able to use the provided `evaluation()` function when calculating the value of board states. Later on, you'll be writing your own, but for now, your minimax will be tested using a provided evaluation function on the autograder.

Note about evaluation functions: call the evaluation function in the following manner: `evaluation(board: chess.board, player: bool)`. Think about which player should be passed into the evaluation function; should it be the player who is currently acting (according to the local max/min node) or should it be in accord with the player at the root of the tree?

If you time out during some of the test cases for this problem, consider adding alpha-beta pruning to your implementation.

In [ ]:
def get_minimax_move(b: chess.Board, evaluation, player: bool, ply: int):
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    max_depth = 2 * ply 
    
    def max_value(board: chess.Board, depth: int, alpha: float, beta: float):
        if depth == 0 or board.is_game_over():
            return evaluation(board, player)
        
        v = -inf
        for move in board.legal_moves:
            board.push(move)
            v = max(v, min_value(board, depth - 1, alpha, beta))
            board.pop()
            
            alpha = max(alpha, v)
            if alpha >= beta: 
                break 
        return v
    
    def min_value(board: chess.Board, depth: int, alpha: float, beta: float):
        if depth == 0 or board.is_game_over():
            return evaluation(board, player)
        
        v = inf
        for move in board.legal_moves:
            board.push(move)
            v = min(v, max_value(board, depth - 1, alpha, beta))
            board.pop()
            
            beta = min(beta, v)
            if beta <= alpha: 
                break 
        return v
    
    best_move = None 
    
    if b.turn == player: 
        best_score = -inf
        for move in b.legal_moves:
            b.push(move)
            # run the minimax algorithm
            score = min_value(b, max_depth - 1, -inf, inf)
            b.pop() 
            
            if score > best_score or best_move is None : 
                best_score = score 
                best_move = move 
                
    else: # root is min node (just inverting everything again)
        best_score = inf
        for move in b.legal_moves:
            b.push(move)
            # run the minimax algorithm
            score = max_value(b, max_depth - 1, -inf, inf)
            b.pop() 
            
            if score < best_score or best_move is None : 
                best_score = score 
                best_move = move 
    
    if best_move is None:
        raise ValueError("No valid moves here")
    
    return best_move
        

## Q2

Implement the expectimax algorithm that chooses the best move for a given board position, arbitrary evaluation function, player colour, and depth.

Make sure to write your implementation to be able to use the provided `evaluation()` function when calculating the value of board states. Later on, you'll be writing your own, but for now, your expectimax will be tested using a provided function on the autograder.

You should handle the evaluation function here in the same way as you did in your minimax implementation: `evaluation(board: chess.board, player: bool)`

In [3]:
def get_expectimax_move(b: chess.Board, evaluation, player: bool, ply: int):   
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    # computing average score under optimal play 
    max_depth = 2 * ply 
    
    def value(board: chess.Board, depth: int) -> float:
        # terminal test 
        if depth == 0 or board.is_game_over():
            return evaluation(board, player)
        
        if board.turn == player: 
            return maxValue(board, depth)
        else:
            return expValue(board, depth)
    
    def maxValue(board: chess.Board, depth: int) -> float:
        v = -inf 
        for move in board.legal_moves:
            board.push(move)
            v = max(v, value(board, depth - 1))
            board.pop()
        return v
    
    def expValue(board: chess.Board, depth: int) -> float:
        moves = list(board.legal_moves)
        
        if not moves: 
            return evaluation(board, player)
        
        p = 1.0/ len(moves)
        v = 0.0 
        
        for move in moves:
            board.push(move)
            v += p * value(board, depth - 1)
            board.pop() 
            
        return v
    
    # root decision 
    best_move = None 
    best_score = -inf 
    for move in b.legal_moves:
        b.push(move)
        score = value(b, max_depth - 1)
        b.pop()
        
        if score > best_score or best_move is None:
            best_score = score
            best_move = move 
        
    if best_move is None:
        raise ValueError("No legal moves available here")
    
    return best_move
        

## Part 2

You will need to complete the implementation of your minimax function before continuing, as the puzzles that follow depend on its correctness.

In the previous questions, you wrote your implementation of minimax and tested it using a hidden evaluation function we wrote for you. Now, you are tasked with developing an original evaluation function `get_position_score` that will be used in conjunction with your minimax function. 

There are several chess puzzles below (drawn from `lichess.org`). Iterate on your evaluation function so that it can solve each puzzle as you come to it. During grading for each puzzle, you must solve the visible case as well as one hidden case to receive full credit. A general solution should be able to solve both.

In [4]:
def get_position_score(board: chess.Board, player: bool) -> float:   
    """
    This function returns a score for a board position, from the given player's point of view.
    A higher score should be more favourable than a low score.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    
    Returns:
    A numerical value representing the score of the board position from player's point of view.
    """
    raise NotImplementedError
    PIECE_VALUES = {
        chess.PAWN: 100, 
        chess.KNIGHT: 320, 
        chess.BISHOP: 330,
        chess.ROOK: 500,
        chess.QUEEN: 900,
        chess.KING: 0,
    }
    
    CENTER_SQUARES = {chess.D4, chess.E4, chess.D5, chess.E5}
    
    if board.is_checkmate():
        if board.turn == player:
            return -1000000
        else:
            return 1000000
    
    if board.is_stalemate() or board.is_insufficient_material() or board.can_claim_draw():
        return 0
    
    def material_and_activity(color: bool) -> int:
        score = 0
        
        # material 
        for ptype, val in PIECE_VALUES.items():
            score += val * (len(board.pieces(ptype, color)) - len(board.pieces(ptype, not color)))
        
        # knits in the center 
        for sq in board.pieces(chess.KNIGHT, color): 
            score += 15 if sq in CENTER_SQUARES else 0
        #reward bishops being slightly off rank 
        for sq in board.pieces(chess.BISHOP, color):
            rank = chess.square_rank(sq)
            score += 5 * (rank if color == chess.WHITE else (7 - rank))
            
        # rooks on open-ish rows / columns 
        for sq in board.pieces(chess.ROOK, color): 
            file_idx = chess.square_file(sq)
            file_squares = [chess.square(file_idx, r) for r in range(8)]
            own_pawn_on_file = any((fsq in board.pieces(chess.PAWN, color)) for fsq in file_squares)
            opp_pawn_on_file = any((fsq in board.pieces(chess.PAWN, not color)) for fsq in file_squares)
            if not own_pawn_on_file and not opp_pawn_on_file:
                score += 20
            elif not own_pawn_on_file and opp_pawn_on_file:
                score += 10 
        return score 
    return material_and_activity(player)
    

### Q3

Given the puzzle below, modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, it will return a move resulting in checkmate.

<img src="images/asgn2%20q3%20puzzle.png" alt="assignment 2 question 3 puzzle figure" width="400"/>

In [5]:
# Feel free to use this code to test your solution to the puzzle above.
puzzle1_board = chess.Board("1rr5/p1p2Rpp/2Qpk3/4n1q1/4P3/8/PPP3PP/R6K")
move = get_minimax_move(puzzle1_board, get_position_score, True, 1)
print(f"Move: {move}")

NotImplementedError: 

In [ ]:
def get_minimax_move(b: chess.Board, evaluation, player: bool, ply: int):
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    max_depth = 2 * ply 
    
    def max_value(board: chess.Board, depth: int, alpha: float, beta: float):
        if depth == 0 or board.is_game_over():
            return float(evaluation(board, player))
        
        v = -math.inf
        for move in board.legal_moves:
            board.push(move)
            v = float(max(float(v), float(min_value(board, depth - 1, alpha, beta))))
            board.pop()
            
            alpha = max(alpha, v)
            if alpha >= beta: 
                break 
        return v
    
    def min_value(board: chess.Board, depth: int, alpha: float, beta: float):
        if depth == 0 or board.is_game_over():
            return float(evaluation(board, player))
        
        v = math.inf
        for move in board.legal_moves:
            board.push(move)
            v = min(float(v), float(max_value(board, depth - 1, alpha, beta)))
            board.pop()
            
            beta = min(beta, v)
            if alpha >= beta: 
                break 
        return v
    
    best_move = None 
    
    if b.turn == player: 
        best_score = -math.inf
        for move in b.legal_moves:
            b.push(move)
            # run the minimax algorithm
            score = min_value(b, max_depth - 1, -math.inf, math.inf)
            b.pop() 
            
            if score > best_score or best_move is None : 
                best_score = score 
                best_move = move 
                
    else: # root is min node (just inverting everything again)
        best_score = math.inf
        for move in b.legal_moves:
            b.push(move)
            # run the minimax algorithm
            score = max_value(b, max_depth - 1, -math.inf, math.inf)
            b.pop() 
            
            if score < best_score or best_move is None : 
                best_score = score 
                best_move = move 
    
    if best_move is None:
        raise ValueError("No valid moves here")
    
    return best_move
        

#### Setting up an Engine

To generate an engine score, first you will need to download the chess engine 'Stockfish' from https://stockfishchess.org/download/.

Once that's done, you may find this docs page useful in setting it up: https://python-chess.readthedocs.io/en/latest/engine.html

Here, the engine score is given in 'Centipawns.' Modify the code below to match your own system and to confirm you have the engine set up correctly.

Note: you may notice that the scores returned by the engine are not consistent. That is, the same position may return a slightly different score. However, because the difference is small, it should not be an issue as the general trends hold true and the puzzles here tend to have an unambiguous best answer. As a failsafe, if your implementation returns the move we're looking for, you'll get full credit regardless of whatever score the engine returns.

In [ ]:
import chess.engine
engine = chess.engine.SimpleEngine.popen_uci(r"stockfish-windows-x86-64-avx2.exe") 
# engine = chess.engine.SimpleEngine.popen_uci(r"/usr/games/stockfish")

# A completely standard board. White starts at a slight advantage, having first move.
board1 = chess.Board()
info1 = engine.analyse(board1, chess.engine.Limit(time=0.1))
print(f"Score: {info1['score'].white()}")

# A standard board, but White is missing both rooks. Naturally, White should have the disadvantage now.
board2 = chess.Board("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/1NBQKBN1")
info2 = engine.analyse(board2, chess.engine.Limit(time=0.1))
print(f"Score: {info2['score'].white()}")

engine.quit()

FileNotFoundError: [Errno 2] No such file or directory: 'stockfish-windows-x86-64-avx2.exe'

#### Q4
Given the puzzle below (White to move), modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, will return a good continuation to the game position. 

You cannot use the engine itself in your evaluation function, but you may find it helpful as you figure out what could make for good features in your evaluation function. Also recommended are the chess position analysis tools available online, such as `https://lichess.org/analysis/standard` to help you get some intuition about what sort of actions may be desirable. What kind of factors would you imagine go into scoring a chess board position? Try to implement some of those in your `get_position_score` function. 

<img src="images/asgn2%20q4%20puzzle.png" alt="assignment 2 question 34 puzzle figure" width="400"/>

In [ ]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle2_board = chess.Board("8/1r3k2/2r1ppp1/8/5PB1/4P3/4PK2/5R2")
engine = chess.engine.SimpleEngine.popen_uci(r"stockfish-windows-x86-64-avx2.exe") 
# engine = chess.engine.SimpleEngine.popen_uci(r"/usr/games/stockfish")
pre_move_info = engine.analyse(puzzle2_board, chess.engine.Limit(time=0.1))

move = get_minimax_move(puzzle2_board, get_position_score, True, 3)
print(f"Move: {move}")
puzzle2_board.push(move)

post_move_info = engine.analyse(puzzle2_board, chess.engine.Limit(time=0.1))
engine.quit()
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")

KeyboardInterrupt: 

#### Q5
Given the puzzle below (White to move), modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, will return a good continuation to the game position. 

You cannot use the engine itself in your evaluation function, but you may find it helpful as you figure out what could make for good features in your evaluation function. Also recommended are the chess position analysis tools available online, such as `https://lichess.org/analysis/standard` to help you get some intuition about what sort of actions may be desirable. What kind of factors would you imagine go into scoring a chess board position? Try to implement some of those in your `get_position_score` function. 

<img src="images/asgn2%20q5%20puzzle.png" alt="assignment 2 question 5 puzzle figure" width="400"/>

In [ ]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle3_board = chess.Board("1k6/2b2p2/2p1p3/1pP2p2/1P1P1P2/8/2N1P3/1K6")
engine = chess.engine.SimpleEngine.popen_uci(r"stockfish-windows-x86-64-avx2.exe")
pre_move_info = engine.analyse(puzzle3_board, chess.engine.Limit(time=0.1))

move = get_minimax_move(puzzle3_board, get_position_score, True, 1)
print(f"Move: {move}")
puzzle3_board.push(move)

post_move_info = engine.analyse(puzzle3_board, chess.engine.Limit(time=0.1))
engine.quit()
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")